<a href="https://colab.research.google.com/github/vsnupoudel/Flask/blob/master/LfTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
!mkdir -p '/content/data/'
%cd /content/data/

/content/data


In [3]:
from google.colab import files
uploaded = files.upload()

In [4]:
train= pd.read_csv('/content/data/training.csv', delimiter=';', decimal=',')
test= pd.read_csv('/content/data/validation.csv', delimiter=';', decimal=',')
# random.seed(12)
train.sample(3)

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,classLabel
59,b,47.67,0.000029,u,-3.095015,0.808325,15.000,t,t,20,f,g,0.0,15000,0.0,t,1,yes.
1817,a,20.67,0.000183,u,4.289618,0.666588,2.085,t,t,5,f,g,220.0,2503,2200000.0,NaN,1,yes.
1204,b,21.25,0.000150,u,-3.159203,0.707669,1.500,f,f,0,f,g,150.0,8,1500000.0,NaN,1,yes.


In [5]:
test.sample(3)

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,classLabel
67,b,16.25,0.000000,y,-4.677870,0.706444,0.250,f,f,0,f,g,60.0,0,600000.0,f,0,no.
157,b,36.17,0.000550,u,-0.866553,0.547565,5.000,f,f,0,f,g,210.0,687,2100000.0,NaN,0,no.
111,b,27.25,0.000167,u,-3.403563,0.818612,5.085,t,t,9,f,g,399.0,827,3990000.0,NaN,0,yes.


In [6]:
train.to_csv('/content/data/train.csv', index= False)
test.to_csv('/content/data/test.csv', index= False)


# Extending pandas dataframe

In [7]:
import pandas as pd
class HLSpd():
  def __init__(self, csvInput):
    self.df = pd.read_csv(csvInput)
      
  def print_stats_numeric(self):
      return self.df.describe(include= np.number).transpose() #.to_json
      
  def print_stats_object(self):
      return self.df.describe(include= np.object).transpose()

In [8]:
tt = HLSpd('/content/data/train.csv')

In [9]:
tt.print_stats_numeric()

,count,mean,std,min,25%,50%,75%,max
v2,3661.0,3.282071e+01,1.266618e+01,13.750000,23.000000,2.867000e+01,4.083000e+01,8.025000e+01
v3,3700.0,5.847872e-04,5.402007e-04,0.000000,0.000150,4.250000e-04,9.625000e-04,2.800000e-03
v5,3700.0,-5.331808e-02,2.871240e+00,-4.996939,-2.516349,-5.278621e-02,2.406251e+00,4.999274e+00
v6,3700.0,7.841879e-01,1.365138e-01,0.273945,0.693130,7.878424e-01,8.760522e-01,1.215066e+00
v7,3700.0,3.439496e+00,4.335229e+00,0.000000,0.500000,1.750000e+00,5.000000e+00,2.850000e+01
v10,3700.0,4.160000e+00,6.750553e+00,0.000000,0.000000,2.000000e+00,6.000000e+00,6.700000e+01
v13,3600.0,1.626950e+02,1.560457e+02,0.000000,0.000000,1.200000e+02,2.800000e+02,1.160000e+03
v14,3700.0,2.246706e+03,8.708571e+03,0.000000,0.000000,1.130000e+02,1.059750e+03,1.000000e+05
v15,3600.0,1.626950e+06,1.560457e+06,0.000000,0.000000,1.200000e+06,2.800000e+06,1.160000e+07
v17,3700.0,9.254054e-01,2.627716e-01,0.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00


In [10]:
tt.print_stats_object()

,count,unique,top,freq
v1,3661,2,b,2382
v4,3636,3,u,3055
v8,3700,2,t,3171
v9,3700,2,t,2239
v11,3700,2,f,1924
v12,3700,3,g,3430
v16,1555,2,t,1023
classLabel,3700,2,yes.,3424


# Initial observations from Pandas' describe()
- We do have missing values both in numeric and categorical columns; will try to use K neareset neighbour to fil missing values. Will ignore v16 completely as it has more than % missing
- Categorical columns have 2 or 3 unique labels only, one hot encoding will be necessary before feeding them to models.
- Numeric values have different distributions, so  a form of standardisation will be necessary. Will try standard scaler.. 

- Also, the classLabel is imbalanced, there are more yes entries than no. Choosing one of the f1 score type metrics will be essential


# Using TFDV for quick visualization
- Installation wll take upto 4 minutes

In [11]:
# !pip install tensorflow_data_validation
# !pip install apache-beam[interactive]

In [12]:
TRAIN_LOCATION = f'/content/data/train.csv'
TEST_LOCATION = f'/content/data/test.csv'

In [13]:
import tensorflow_data_validation as tfdv

stats_train = tfdv.generate_statistics_from_csv(data_location= TRAIN_LOCATION)
stats_test = tfdv.generate_statistics_from_csv(data_location= TEST_LOCATION)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Visualize just the train statistics for now.
- This satisfies most of what is asked in the assingmnent, except outlier detection. ( Outlier can be detected from histogram to the right also)
- Box plots / Violin plots are better for outlier detection

In [14]:
tfdv.visualize_statistics( stats_train )

# To do : Violin plot for better outlier detection

In [15]:
import seaborn as sns

# To do: Feature Selection and/or Feature Engineering
- Skipping for now as I plan to use Neural network and Keras Tuner. DNN is in a way automated feature Engineering/Selection

- https://scikit-learn.org/stable/modules/feature_selection.html

Preprocessing function for inputs similar to the data provided( Semicolon separated file, with comma as decimal)

**Fit imputer with training data**

In [16]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [17]:
TRAIN_LOCATION = f'/content/data/training.csv'
TEST_LOCATION = f'/content/data/validation.csv'


In [18]:
#  pd.read_csv( TEST_LOCATION , delimiter=';', decimal=',')['classLabel'].unique()

**Function to replace 'no.' or 'yes.' to 0 and 1 respectively**

In [19]:
label_diction = { 'no.': 0 , 'yes.':1, 'yes':1, 'no':0}
def load_labels(inputCsv):
  return pd.read_csv( inputCsv , delimiter=';', decimal=',')['classLabel'].str.lower().replace(to_replace= label_diction)

In [20]:
 def load_into_dataframe(inputCsv) :
  df = pd.read_csv( inputCsv , delimiter=';', decimal=',')  # load as dataframe
  df = df.drop(columns= ['v16','classLabel'])  #drop the column that has 50% training
  global COLS
  COLS = df.columns
  return df

In [21]:
def one_hot_encode(dfin):
  # dfin = pd.DataFrame(dfin, columns= COLS )
  return pd.get_dummies(dfin
                 , columns= ['v1','v4','v8','v9','v11','v12']
                 , drop_first=False) # false as other values might be in test set or later

In [22]:
pipe = Pipeline([
                  (  'load_into_df', FunctionTransformer( load_into_dataframe )  )                     
                    , ( 'one_hot_enc' ,  FunctionTransformer( one_hot_encode ) )     
                    , ('imputer', KNNImputer(n_neighbors=3) )     
                    , ('scaler', StandardScaler()  )          
                    ])

In [23]:
# df = one_hot_encode( load_into_dataframe(TRAIN_LOCATION) )
# ddf = KNNImputer().fit_transform(df)

In [24]:
pfit = pipe.fit( TRAIN_LOCATION )
X_train = pfit.transform( TRAIN_LOCATION)
X_test = pfit.transform( TEST_LOCATION )

In [25]:
y_train = load_labels(TRAIN_LOCATION)
y_test = load_labels(TEST_LOCATION)

# Train a DNN with Keras and Keras tuner

In [45]:
# pip install keras-tuner --upgrade

In [68]:
import keras_tuner as kt
from tensorflow import keras
import numpy as np
import tensorflow as tf

In [87]:
def build_model(hp):
  model = keras.Sequential()
  model.add(keras.layers.Dense(
      hp.Choice('dense1',  list(range(8,32,4))  ),
      activation='relu'))
  
  model.add(keras.layers.Dropout(0.5))
  
  model.add(keras.layers.Dense(
      hp.Choice('dense2',  list(range(4,12,4))  ),
      activation='tanh'))
  
  model.add(keras.layers.Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', metrics = [keras.metrics.AUC()
                  , 'accuracy'])
  return model

In [88]:
tuner = kt.RandomSearch(
            build_model,
            objective='val_loss',
            max_trials=25,
            directory = '/content/data')

INFO:tensorflow:Reloading Oracle from existing project /content/data/untitled_project/oracle.json


INFO:tensorflow:Reloading Oracle from existing project /content/data/untitled_project/oracle.json


In [89]:
from sklearn.model_selection import train_test_split
# X_tr, y_tr, X_val , y_val = train_test_split( X_train, y_train, test_size= 0.2, stratify= y_train)

In [90]:
tuner.search(X_train, y_train, epochs= 25, validation_split = 0.25 ) # try validation percentage

Trial 10 Complete [00h 00m 11s]
val_loss: 0.0004229855549056083

Best val_loss So Far: 8.034301572479308e-05
Total elapsed time: 00h 01m 31s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [91]:
tuner.results_summary()

Results summary
Results in /content/data/untitled_project
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
dense1: 24
dense2: 8
Score: 8.034301572479308e-05
Trial summary
Hyperparameters:
dense1: 20
dense2: 8
Score: 8.726548549020663e-05
Trial summary
Hyperparameters:
dense1: 16
dense2: 8
Score: 0.00014577469846699387
Trial summary
Hyperparameters:
dense1: 28
dense2: 4
Score: 0.0001484658569097519
Trial summary
Hyperparameters:
dense1: 20
dense2: 4
Score: 0.0002825883566401899
Trial summary
Hyperparameters:
dense1: 24
dense2: 4
Score: 0.00029054819606244564
Trial summary
Hyperparameters:
dense1: 12
dense2: 8
Score: 0.0004229855549056083
Trial summary
Hyperparameters:
dense1: 16
dense2: 4
Score: 0.0004893004079349339
Trial summary
Hyperparameters:
dense1: 8
dense2: 8
Score: 0.0021568527445197105
Trial summary
Hyperparameters:
dense1: 8
dense2: 4
Score: 0.002446168102324009


In [92]:
# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model.
loss, auc, accuracy = best_model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 2ms/step - loss: 4.4696 - auc: 0.4955 - accuracy: 0.4950


Removing directory works

In [102]:
!rm -rf '/content/data/untitled_project'

In [94]:
y_test.value_counts()

0    107
1     93
Name: classLabel, dtype: int64

In [95]:
y_train.value_counts()

1    3424
0     276
Name: classLabel, dtype: int64

## Test accuracy is bleak, sometimes worse than random guess ( 50%)
- But training is 100%. Major overfitting going on

In [100]:
from sklearn.metrics import confusion_matrix

confusion_matrix( y_test , best_model.predict(X_test)>=0.5 )

array([[53, 54],
       [47, 46]])

In [101]:
confusion_matrix( y_train , best_model.predict(X_train)>=0.5 )

array([[ 276,    0],
       [   0, 3424]])